In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/100-most-watched-anime-in-the-world/most_watched_anime_dataset_100_entries.csv


In [2]:
import warnings
warnings.filterwarnings("ignore")

I've found this dataset on kaggle with information on the most anime watched in the world. As an avid anime fan, I am curious which anime really are the most popular. In addition, I want to see if there are major differences in the most viewed anime in different countries. It will also be interesting to separate anime by categories to see which anime is the most popular in each genre.
First I will load the dataset and clean up the data for visualizing.

In [3]:
df = pd.read_csv("/kaggle/input/100-most-watched-anime-in-the-world/most_watched_anime_dataset_100_entries.csv")

In [4]:
 df.head()

,Anime Name,Most Watched in Country,Ratings,Number of Episodes,Animation Studio Name,Budget (in Million USD),Release Year,Genre,Duration per Episode (minutes)
0,Fullmetal Alchemist,Brazil,8.8,317.0,Ufotable,80.61,1998.0,Fantasy,NaN
1,Haikyuu!!,Mexico,9.2,420.0,MAPPA,74.99,2022.0,Adventure,59.0
2,NaN,South Korea,7.4,419.0,A-1 Pictures,59.05,2009.0,Mystery,56.0
3,Bleach,Brazil,6.4,NaN,Ufotable,45.35,2002.0,Fantasy,55.0
4,Sword Art Online,NaN,9.8,327.0,Madhouse,15.90,2017.0,NaN,43.0


I want to focus primarily on ratings

In [5]:
df.isnull().sum()

Anime Name                        10
Most Watched in Country           10
Ratings                           10
Number of Episodes                10
Animation Studio Name             10
Budget (in Million USD)           10
Release Year                      10
Genre                             10
Duration per Episode (minutes)    10
dtype: int64

In [6]:
df.dropna(inplace=True)

In [7]:
df.shape

(41, 9)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 41 entries, 1 to 99
Data columns (total 9 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Anime Name                      41 non-null     object 
 1   Most Watched in Country         41 non-null     object 
 2   Ratings                         41 non-null     float64
 3   Number of Episodes              41 non-null     float64
 4   Animation Studio Name           41 non-null     object 
 5   Budget (in Million USD)         41 non-null     float64
 6   Release Year                    41 non-null     float64
 7   Genre                           41 non-null     object 
 8   Duration per Episode (minutes)  41 non-null     float64
dtypes: float64(5), object(4)
memory usage: 3.2+ KB


In [9]:
df.describe()

,Ratings,Number of Episodes,Budget (in Million USD),Release Year,Duration per Episode (minutes)
count,41.000000,41.000000,41.000000,41.000000,41.000000
mean,8.158537,252.902439,73.957317,2004.292683,38.146341
std,1.174729,149.605950,44.068149,9.935401,11.332610
min,6.300000,12.000000,5.820000,1990.000000,21.000000
25%,7.100000,114.000000,44.880000,1995.000000,30.000000
50%,8.100000,250.000000,61.380000,2002.000000,37.000000
75%,9.400000,381.000000,110.820000,2013.000000,45.000000
max,9.900000,491.000000,149.390000,2022.000000,59.000000


We need to install altair and load the necessary libraries for altering the data and creating visualizations

In [10]:
!pip install altair -- upgrade
!pip install vega_datasets

ERROR: Could not find a version that satisfies the requirement upgrade (from versions: none)
ERROR: No matching distribution found for upgrade


In [11]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import altair as alt

In [12]:
df['Anime Name'].unique().shape

(20,)

In [13]:
 pip install -U kaleido

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 21.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


Let's create a visualization to view the number of episodes in each anime. I will use a bar chart to easily compare the anime to eachother

In [14]:
#interactive
hover = alt.selection_single(
    on = 'mouseover',
    fields = ['Anime Name'],
    empty = 'none'
)

# Create the main chart with hover selection
chart = alt.Chart(df).mark_bar().encode(
    y = alt.Y('Anime Name:N', sort='-x', title='Anime Name'),
    x = alt.X('Number of Episodes:Q', title='Number of Episodes'),
    color = alt.Color('Number of Episodes:Q', scale=alt.Scale(scheme='blues')),
    opacity = alt.condition(hover, alt.value(1.0), alt.value(0.6)),
    tooltip = ['Anime Name', 'Number of Episodes']
).properties(
    title = 'Number of Episodes per Anime',
    width = 600,
    height = 20 * len(df)  # Dynamic height based on number of anime
).add_selection(
    hover
)

# Add text labels
text = chart.mark_text(
    align = 'left',
    baseline = 'middle',
    dx = 3  # Slight offset from the end of the bar
).encode(
    text = 'Number of Episodes:Q'
)

# Combine chart and text
final_chart = (chart + text).configure_view(
    strokeWidth = 0
).configure_axis(
    labelFontSize = 12,
    titleFontSize = 14
).configure_title(
    fontSize = 16
)

# Display the chart
final_chart

alt.LayerChart(...)

Comparing budget and rating

In [15]:
fig = px.density_contour(df, x="Ratings", y="Budget (in Million USD)", 
                         title="📈 KDE Density Plot: Ratings vs Budget")

fig.update_layout(
    xaxis_title="Ratings",
    yaxis_title="Budget (Million USD)",
    template="plotly_dark"
)

fig.show(renderer = "iframe")
fig.write_image("plotly_chart.png")

Decided to focus on ratings for anime. Create a scatter matrix to compare ratings to other values

In [16]:
fig = px.scatter_matrix(df, 
                        dimensions=['Ratings','Number of Episodes', 'Budget (in Million USD)', 'Release Year'],
                        color='Ratings',  
                        title="📊 Pairplot of Anime Dataset",
                        template="plotly_dark")

fig.update_layout(
    font=dict(size=12),
    width=900, height=900
)

fig.show(renderer = "iframe")
fig.write_image("plotly_chart.png")

Create a basic Vis that allows you to select an anime name and display information about it

In [17]:
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display

# Assuming df is already loaded
anime_dropdown = widgets.Dropdown(
    options=df['Anime Name'].unique(),
    description='Anime:',
    disabled=False
)

output = widgets.Output()

def update_graph(change):
    with output:
        output.clear_output()
        selected_anime = anime_dropdown.value
        if not selected_anime:
            display("Select an Anime to Display")
            return
        
        filtered_df = df[df['Anime Name'] == selected_anime]
        
        # Define categories and values
        categories = ["Ratings", "Budget (in Million USD)", "Number of Episodes", "Duration per Episode (minutes)"]
        values = [
            filtered_df.iloc[0]['Ratings'],
            filtered_df.iloc[0]['Budget (in Million USD)'],
            filtered_df.iloc[0]['Number of Episodes'],
            filtered_df.iloc[0]['Duration per Episode (minutes)']
        ]
        
        # Radar chart requires values to be circular
        values.append(values[0])
        categories.append(categories[0])
        
        fig = go.Figure()
        fig.add_trace(go.Scatterpolar(
            r=values,
            theta=categories,
            fill='toself',
            name=selected_anime
        ))
        
        fig.update_layout(
            polar=dict(radialaxis=dict(visible=True)),
            title=f'Radar Chart: {selected_anime}'
        )
        
        fig.show()

anime_dropdown.observe(update_graph, names='value')

display(anime_dropdown, output)



Dropdown(description='Anime:', options=('Haikyuu!!', 'Bleach', 'Tokyo Ghoul', 'Naruto', 'Mob Psycho 100', 'Neo…

Output()

In [18]:
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display

# Assuming df is already loaded
anime_dropdown = widgets.Dropdown(
    options=df['Anime Name'].unique(),
    description='Anime:',
    disabled=False
)

output = widgets.Output()

def update_table(change):
    with output:
        output.clear_output()
        selected_anime = anime_dropdown.value
        if not selected_anime:
            display("Select an Anime to Display")
            return
        
        filtered_df = df[df['Anime Name'] == selected_anime]

        # Extract anime details
        categories = ["Ratings", "Budget (in Million USD)", "Number of Episodes", "Duration per Episode (minutes)"]
        values = [
            filtered_df.iloc[0]['Ratings'],
            filtered_df.iloc[0]['Budget (in Million USD)'],
            filtered_df.iloc[0]['Number of Episodes'],
            filtered_df.iloc[0]['Duration per Episode (minutes)']
        ]

        # Create a table
        fig = go.Figure(data=[go.Table(
            header=dict(values=["Category", "Value"],
                        fill_color="lightblue",
                        align="left"),
            cells=dict(values=[categories, values],
                       fill_color="black",
                       align="left")
        )])

        fig.update_layout(title=f"📋 Details for {selected_anime}")

        fig.show()

anime_dropdown.observe(update_table, names='value')

display(anime_dropdown, output)

Dropdown(description='Anime:', options=('Haikyuu!!', 'Bleach', 'Tokyo Ghoul', 'Naruto', 'Mob Psycho 100', 'Neo…

Output()

Let's look at the highest budget anime

In [19]:
df1 = df.loc[df.groupby("Anime Name")["Budget (in Million USD)"].idxmax()]
df1 = df1[df1["Budget (in Million USD)"] > 50].sort_values(by="Budget (in Million USD)", ascending=False).head(5)

fig = px.bar(df1, x="Anime Name", y="Budget (in Million USD)", 
             title="Most High-Budget Anime",
             text="Budget (in Million USD)",  
             color="Anime Name",  
             color_discrete_sequence=px.colors.qualitative.Set1,
             hover_data={"Ratings": True, "Anime Name": False, "Budget (in Million USD)": False}  # Show only Ratings
            )  

fig.update_traces(texttemplate='%{text:.1f}M', textposition="outside")

fig.update_layout(
    xaxis_title="Anime Name",
    yaxis_title="Budget (Million USD)",
    font=dict(size=13),
    template="plotly_dark", 
    xaxis_tickangle=-15  
)

fig.show(renderer="iframe")
fig.write_image("plotly_chart.png")

Pie Chart

In [20]:
fig = px.pie(df, names="Genre", 
             title="Genre Distribution of Anime",
             color_discrete_sequence=px.colors.qualitative.Set2, 
             hole=0.3)  

fig.update_traces(textinfo="percent+label", 
                  pull=[0.05 if i == df['Genre'].value_counts().idxmax() else 0 for i in df['Genre'].unique()],  # Pops out the most common genre
                  marker=dict(line=dict(color='black', width=1)))  

fig.update_layout(
    showlegend=True,
    font=dict(size=13),
    template="plotly_dark"  
)

fig.show(renderer = "iframe")
fig.write_image("plotly_chart.png")

In [21]:
fig = px.box(df, x="Genre", y="Ratings", 
             title="Rating Distribution by Genre",
             color="Genre",
             color_discrete_sequence=px.colors.qualitative.Set2)

fig.update_layout(
    xaxis_title="Genre",
    yaxis_title="Ratings",
    font=dict(size=13),
    template="plotly_dark",
    xaxis_tickangle=-15  
)

fig.show(renderer="iframe")
fig.write_image("box_plot.png")

Highest rating over time
Looks like a bust
Wait for claude to help

In [22]:
fig = px.scatter(df, x="Budget (in Million USD)", y="Ratings", 
                 color="Genre",  
                 hover_data=["Anime Name", "Animation Studio Name"],  
                 template="plotly_dark",  
                 opacity=0.8)  

fig.update_layout(
    title="Budget vs Ratings of Anime",
    xaxis_title="Budget (Million USD)",
    yaxis_title="Ratings",
    font=dict(size=13),
    legend_title="Genre"
)

fig.show(renderer = "iframe")
fig.write_image("plotly_chart.png")

Create map of the world with most popular anime of each section.

In [23]:
import plotly.express as px

fig = px.choropleth(df, 
                    locations="Most Watched in Country",  
                    locationmode="country names",  
                    color="Anime Name",  # Color based on the most popular anime
                    hover_name="Most Watched in Country",  
                    title="Most Popular Anime by Country",  
                    color_discrete_sequence=px.colors.qualitative.Set3)  # Use distinct colors

fig.update_layout(
    geo=dict(showcoastlines=True, showland=True, landcolor="lightgray"),  
    template="plotly_dark",  
    font=dict(size=12, color="white"),  
    title_font_size=20
)

fig.show(renderer="iframe")
fig.write_image("most_popular_anime_by_country.png")

Lets combine all our plots together into a preliminary visualization